In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("finalDataset.csv")
df.drop_duplicates(inplace=True)
df=df.dropna()
df

,userID,movieID,rating,title,imdbID,keywords
0,8624,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
1,8930,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
2,12028,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
3,13916,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
4,15264,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
...,...,...,...,...,...,...
160066,56620,7032,5.0,Star Wars: Episode III - Revenge of the Sith,121766,"personality change,tragic villain,star wars,ad..."
160183,46840,5328,4.5,Singin' in the Rain,45152,"movie studio,film within a film,femme fatale,h..."
160229,32308,41136,5.0,Taxi Driver,75314,"taxi driver,loner,street life,child prostituti..."
160297,52983,25934,4.5,The Godfather,68646,"mafia,patriarch,crime family,organized crime,g..."


In [3]:
def jaccard_similarity(keywords1, keywords2):
    set1 = set(keywords1.split(','))
    set2 = set(keywords2.split(','))
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

In [4]:
def calculate_similarity(movie_idx1, movie_idx2, dataset):
    keywords1 = dataset.loc[movie_idx1]['keywords']
    keywords2 = dataset.loc[movie_idx2]['keywords']
    return jaccard_similarity(keywords1, keywords2)

In [5]:
movie_idx1 = 0  # Index of the first movie
movie_idx2 = 1  # Index of the second movie
similarity = calculate_similarity(movie_idx1, movie_idx2, df)
print("Jaccard similarity between keywords:", similarity)

# Check the data type of the 'keywords' column
print(df['keywords'].dtype)

Jaccard similarity between keywords: 1.0
object


In [6]:
if df['keywords'].dtype != 'object':
    df['keywords'] = df['keywords'].astype(str)

# Now, split the 'keywords' column into a list of keywords
df['keywords'] = df['keywords'].str.split(',')

In [7]:
unique_keywords = set()
for keywords in df['keywords']:
    unique_keywords.update(keywords)

In [8]:
def jaccard_similarity_list(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [17]:
# Remove duplicates based on columns other than the one with lists
df.drop_duplicates(subset=df.columns.difference(['keywords']), inplace=True)

# Drop rows with missing values
df.dropna(inplace=True)

# Check the structure of the dataframe
print(df.head())


   userID  movieID  rating           title  imdbID  \
0    8624        3     4.5  Grumpy Old Men  107050   
1    8930        3     4.5  Grumpy Old Men  107050   
2   12028        3     4.5  Grumpy Old Men  107050   
3   13916        3     4.5  Grumpy Old Men  107050   
4   15264        3     4.5  Grumpy Old Men  107050   

                                            keywords  
0  [old man, feud, old age, neighbor neighbor rel...  
1  [old man, feud, old age, neighbor neighbor rel...  
2  [old man, feud, old age, neighbor neighbor rel...  
3  [old man, feud, old age, neighbor neighbor rel...  
4  [old man, feud, old age, neighbor neighbor rel...  


In [28]:
def recommend_movies_by_title(seed_movie_title, top_n=5, S=set()):
    # Find the seed movie's ID
    seed_movie_id = df[df['title'] == seed_movie_title]['movieID'].values[0]
    
    # Find the seed movie's keywords
    seed_movie_keywords = df[df['movieID'] == seed_movie_id]['keywords'].values[0]
    
    # Calculate similarity for each movie in S, excluding the seed movie
    similarities = []
    for movie_id in S:
        if movie_id != seed_movie_id:  # Ensure the seed movie is not considered
            movie_keywords = df[df['movieID'] == movie_id]['keywords'].values[0]
            similarity = jaccard_similarity(seed_movie_keywords, movie_keywords)
            if similarity > 0.03:  # Adjusted to correct threshold
                similarities.append((movie_id, similarity))
    
    # Sort by similarity and get the top N
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Fetch the movie titles for the recommended movie IDs, ensuring uniqueness
    recommended_movies = []
    seen_titles = set()
    for movie_id, similarity in similarities:
        movie_title = df[df['movieID'] == movie_id]['title'].values[0]
        if movie_title not in seen_titles:
            recommended_movies.append((movie_title, similarity))
            seen_titles.add(movie_title)
            if len(recommended_movies) == top_n:
                break
    
    return recommended_movies

In [41]:
def interactive_recommendation_system():
    # Initial seed movie title
    seed_movie_title = input("Enter the title of a movie to start with: ")
    
    # Initialize S with items related to the seed item
    seed_movie_ids = set(df[df['title'] == seed_movie_title]['movieID'].values)
    S = set(df['movieID'].values) - seed_movie_ids  # Exclude all instances of the seed movie
    
    while True:
        # Generate recommendations from S
        top_n = 5
        recommended_movies = recommend_movies_by_title(seed_movie_title, top_n, S)
        
        if not recommended_movies:
            print("No more recommendations available.")
            break
        
        # Display recommendations
        print(f"\nRecommendations for '{seed_movie_title}':")
        for idx, (movie_title, similarity) in enumerate(recommended_movies, start=1):
            print(f"{idx}. {movie_title} (Similarity: {similarity})")
        
        # Ask for user selection
        selected_index = int(input("\nSelect a movie from the list above by entering its number: ")) - 1
        
        if selected_index < 0 or selected_index >= len(recommended_movies):
            print("Invalid selection. Please enter a valid number.")
            continue
        
        selected_movie_title = recommended_movies[selected_index][0]
        
        # Remove selected movie from S
        selected_movie_id = df[df['title'] == selected_movie_title]['movieID'].values[0]
        S.remove(selected_movie_id)
        
        # Remove all non-selected movies from S
        non_selected_movie_ids = {df[df['title'] == movie[0]]['movieID'].values[0] for idx, movie in enumerate(recommended_movies) if idx != selected_index}
        S.difference_update(non_selected_movie_ids)
        
        # Ask for continuation
        continue_choice = input("\nDo you want to continue? (yes/no): ")
        
        if continue_choice.lower() == 'no':
            print(f"\nYou selected: {selected_movie_title}")
            break
        else:
            # Use the selected movie as the new seed movie
            seed_movie_title = selected_movie_title

# Start the interactive recommendation system
interactive_recommendation_system()

Enter the title of a movie to start with: The Godfather

Recommendations for 'The Godfather':
1. Goodfellas (Similarity: 0.42857142857142855)
2. Carlito's Way (Similarity: 0.25)
3. Scarface (Similarity: 0.25)
4. Casino (Similarity: 0.1111111111111111)
5. A Bronx Tale (Similarity: 0.1111111111111111)

Select a movie from the list above by entering its number: 5

Do you want to continue? (yes/no): yes

Recommendations for 'A Bronx Tale':
1. Don't Tell Mom the Babysitter's Dead (Similarity: 0.16666666666666666)
2. In the Name of the Father (Similarity: 0.1111111111111111)
3. The Man Without a Face (Similarity: 0.1111111111111111)
4. Searching for Bobby Fischer (Similarity: 0.1111111111111111)
5. The Three Musketeers (Similarity: 0.1111111111111111)

Select a movie from the list above by entering its number: 5

Do you want to continue? (yes/no): yes

Recommendations for 'The Three Musketeers':
1. The Three Musketeers (Similarity: 1.0)
2. DragonHeart (Similarity: 0.1111111111111111)
3. Indi

Smean

In [34]:
import numpy as np

def recommend_movies_by_title(seed_movie_title, top_n=5, S=set(), weights={}):
    # Find the seed movie's ID
    seed_movie_id = df[df['title'] == seed_movie_title]['movieID'].values[0]
    
    # Find the seed movie's keywords
    seed_movie_keywords = df[df['movieID'] == seed_movie_id]['keywords'].values[0]
    
    # Calculate similarity for each movie in S, excluding the seed movie
    similarities = []
    for movie_id in S:
        if movie_id != seed_movie_id:  # Ensure the seed movie is not considered
            movie_keywords = df[df['movieID'] == movie_id]['keywords'].values[0]
            base_similarity = jaccard_similarity(seed_movie_keywords, movie_keywords)
            # Adjust similarity based on weight
            adjusted_similarity = base_similarity + weights.get(movie_id, 1)  # Default weight is 1
            if base_similarity > 0.03:  # Adjusted to correct threshold
                similarities.append((movie_id, adjusted_similarity))
    
    # Sort by adjusted similarity and get the top N
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Fetch the movie titles for the recommended movie IDs, ensuring uniqueness
    recommended_movies = []
    seen_titles = set()
    for movie_id, similarity in similarities:
        movie_title = df[df['movieID'] == movie_id]['title'].values[0]
        if movie_title not in seen_titles:
            recommended_movies.append((movie_title, similarity))
            seen_titles.add(movie_title)
            if len(recommended_movies) == top_n:
                break
    
    return recommended_movies

def update_weights(selected_movie_id, S, weights):
    selected_keywords = df[df['movieID'] == selected_movie_id]['keywords'].values[0]
    
    # Calculate similarity of the selected movie with all other movies
    similarity_with_selected = {}
    for movie_id in S:
        if movie_id != selected_movie_id:
            movie_keywords = df[df['movieID'] == movie_id]['keywords'].values[0]
            similarity_with_selected[movie_id] = jaccard_similarity(selected_keywords, movie_keywords)
    
    # Calculate mean similarity of selected movie with non-selected movies
    non_selected_similarities = [similarity for movie_id, similarity in similarity_with_selected.items()]
    mean_similarity = np.mean(non_selected_similarities) if non_selected_similarities else 0
    
    # Update weights
    for movie_id, similarity in similarity_with_selected.items():
        weights[movie_id] = weights.get(movie_id, 1) + similarity - mean_similarity

def interactive_recommendation_system():
    seed_movie_title = input("Enter the title of a movie to start with: ")
    
    # Initialize S with items related to the seed item and weights
    seed_movie_ids = set(df[df['title'] == seed_movie_title]['movieID'].values)
    S = set(df['movieID'].values) - seed_movie_ids  # Exclude all instances of the seed movie
    weights = {movie_id: 1 for movie_id in S}  # Initialize weights
    
    while True:
        recommended_movies = recommend_movies_by_title(seed_movie_title, 5, S, weights)
        
        if not recommended_movies:
            print("No more recommendations available.")
            break
        
        print(f"\nRecommendations for '{seed_movie_title}':")
        for idx, (movie_title, similarity) in enumerate(recommended_movies, start=1):
            print(f"{idx}. {movie_title} (Similarity: {similarity})")
        
        selected_index = int(input("\nSelect a movie from the list above by entering its number: ")) - 1
        selected_movie_title = recommended_movies[selected_index][0]
        selected_movie_id = df[df['title'] == selected_movie_title]['movieID'].values[0]
        
        # Update weights based on the selection
        update_weights(selected_movie_id, S, weights)
        
        # Remove selected and non-selected movies from S
        S.remove(selected_movie_id)
        non_selected_movie_ids = {movie_id for movie_id in S if movie_id != selected_movie_id}
        S.difference_update(non_selected_movie_ids)
        
        continue_choice = input("\nDo you want to continue? (yes/no): ")
        
        if continue_choice.lower() == 'no':
            print(f"\nYou selected: {selected_movie_title}")
            break
        else:
            seed_movie_title = selected_movie_title

# Start the interactive recommendation system
interactive_recommendation_system()


Enter the title of a movie to start with: The Godfather

Recommendations for 'The Godfather':
1. Goodfellas (Similarity: 1.4285714285714286)
2. Carlito's Way (Similarity: 1.25)
3. Scarface (Similarity: 1.25)
4. Casino (Similarity: 1.1111111111111112)
5. A Bronx Tale (Similarity: 1.1111111111111112)

Select a movie from the list above by entering its number: 4

Do you want to continue? (yes/no): yes
No more recommendations available.


In [35]:
def recommend_movies_by_title(seed_movie_title, top_n=5, S=set(), weights={}):
    # Find the seed movie's ID
    seed_movie_id = df[df['title'] == seed_movie_title]['movieID'].values[0]
    
    # Find the seed movie's keywords
    seed_movie_keywords = df[df['movieID'] == seed_movie_id]['keywords'].values[0]
    
    # Calculate similarity for each movie in S, excluding the seed movie
    similarities = []
    for movie_id in S:
        if movie_id != seed_movie_id:  # Ensure the seed movie is not considered
            movie_keywords = df[df['movieID'] == movie_id]['keywords'].values[0]
            similarity = jaccard_similarity(seed_movie_keywords, movie_keywords)
            if similarity > 0.03:  # Adjusted to correct threshold
                # Adjust similarity based on weights
                adjusted_similarity = similarity + weights.get(movie_id, 0)
                similarities.append((movie_id, adjusted_similarity))
    
    # Sort by adjusted similarity and get the top N
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Fetch the movie titles for the recommended movie IDs, ensuring uniqueness
    recommended_movies = []
    seen_titles = set()
    for movie_id, similarity in similarities:
        movie_title = df[df['movieID'] == movie_id]['title'].values[0]
        if movie_title not in seen_titles:
            recommended_movies.append((movie_title, similarity))
            seen_titles.add(movie_title)
            if len(recommended_movies) == top_n:
                break
    
    return recommended_movies

def interactive_recommendation_system():
    # Initialize weights for each item to 1
    weights = {}
    
    # Initial seed movie title
    seed_movie_title = input("Enter the title of a movie to start with: ")
    
    # Initialize S with items related to the seed item
    seed_movie_ids = set(df[df['title'] == seed_movie_title]['movieID'].values)
    S = set(df['movieID'].values) - seed_movie_ids  # Exclude all instances of the seed movie
    
    while True:
        # Generate recommendations from S using weights
        top_n = 5
        recommended_movies = recommend_movies_by_title(seed_movie_title, top_n, S, weights)
        
        if not recommended_movies:
            print("No more recommendations available.")
            break
        
        # Display recommendations
        print(f"\nRecommendations for '{seed_movie_title}':")
        for idx, (movie_title, similarity) in enumerate(recommended_movies, start=1):
            print(f"{idx}. {movie_title} (Similarity: {similarity})")
        
        # Ask for user selection
        selected_index = int(input("\nSelect a movie from the list above by entering its number: ")) - 1
        
        if selected_index < 0 or selected_index >= len(recommended_movies):
            print("Invalid selection. Please enter a valid number.")
            continue
        
        selected_movie_title = recommended_movies[selected_index][0]
        
        # Remove selected movie from S
        selected_movie_id = df[df['title'] == selected_movie_title]['movieID'].values[0]
        S.remove(selected_movie_id)
        
        # Calculate mean similarity between selected item and non-selected items
        non_selected_similarities = [similarity for movie_title, similarity in recommended_movies if movie_title != selected_movie_title]
        mean_similarity = sum(non_selected_similarities) / len(non_selected_similarities) if non_selected_similarities else 0
        
        # Update weights for each item in S
        for movie_id, similarity in recommended_movies:
            weights[movie_id] = weights.get(movie_id, 1) + similarity - mean_similarity
        
        # Ask for continuation
        continue_choice = input("\nDo you want to continue? (yes/no): ")
        
        if continue_choice.lower() == 'no':
            print(f"\nYou selected: {selected_movie_title}")
            break
        else:
            # Use the selected movie as the new seed movie
            seed_movie_title = selected_movie_title

# Start the interactive recommendation system
interactive_recommendation_system()


Enter the title of a movie to start with: The Godfather

Recommendations for 'The Godfather':
1. Goodfellas (Similarity: 0.42857142857142855)
2. Carlito's Way (Similarity: 0.25)
3. Scarface (Similarity: 0.25)
4. Casino (Similarity: 0.1111111111111111)
5. A Bronx Tale (Similarity: 0.1111111111111111)

Select a movie from the list above by entering its number: 4

Do you want to continue? (yes/no): yes

Recommendations for 'Casino':
1. Fear and Loathing in Las Vegas (Similarity: 0.25)
2. Get Shorty (Similarity: 0.1111111111111111)
3. Leaving Las Vegas (Similarity: 0.1111111111111111)
4. A Bronx Tale (Similarity: 0.1111111111111111)
5. Carlito's Way (Similarity: 0.1111111111111111)

Select a movie from the list above by entering its number: 5

Do you want to continue? (yes/no): yes

Recommendations for 'Carlito's Way':
1. Romeo + Juliet (Similarity: 0.25)
2. Goodfellas (Similarity: 0.25)
3. Boogie Nights (Similarity: 0.25)
4. A History of Violence (Similarity: 0.25)
5. Saturday Night Fever

In [36]:


def recommend_movies_by_title(seed_movie_title, top_n=5, S=set(), weights=None):
    # Find the seed movie's ID
    seed_movie_id = df[df['title'] == seed_movie_title]['movieID'].values[0]
    
    # Find the seed movie's keywords
    seed_movie_keywords = df[df['movieID'] == seed_movie_id]['keywords'].values[0]
    
    # Calculate similarity for each movie in S, excluding the seed movie
    similarities = []
    for movie_id in S:
        if movie_id != seed_movie_id:  # Ensure the seed movie is not considered
            movie_keywords = df[df['movieID'] == movie_id]['keywords'].values[0]
            similarity = jaccard_similarity(seed_movie_keywords, movie_keywords)
            if similarity > 0.03:  # Adjusted to correct threshold
                similarities.append((movie_id, similarity))
    
    # Sort by similarity and get the top N
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Update weights based on similarities with selected movie
    if weights:
        for movie_id, similarity in similarities:
            if movie_id in weights:
                mean_similarity = sum([s for _, s in similarities]) / len(similarities)
                delta_wi = similarity - mean_similarity
                weights[movie_id] += delta_wi
    
    # Fetch the movie titles for the recommended movie IDs, ensuring uniqueness
    recommended_movies = []
    seen_titles = set()
    for movie_id, similarity in similarities:
        movie_title = df[df['movieID'] == movie_id]['title'].values[0]
        if movie_title not in seen_titles:
            recommended_movies.append((movie_title, similarity))
            seen_titles.add(movie_title)
            if len(recommended_movies) == top_n:
                break
    
    return recommended_movies, weights

def interactive_recommendation_system():
    # Initial seed movie title
    seed_movie_title = input("Enter the title of a movie to start with: ")
    
    # Initialize S with items related to the seed item
    seed_movie_ids = set(df[df['title'] == seed_movie_title]['movieID'].values)
    S = set(df['movieID'].values) - seed_movie_ids  # Exclude all instances of the seed movie
    
    # Initialize weights
    weights = {movie_id: 1 for movie_id in S}
    
    while True:
        # Generate recommendations from S
        top_n = 5
        recommended_movies, weights = recommend_movies_by_title(seed_movie_title, top_n, S, weights)
        
        if not recommended_movies:
            print("No more recommendations available.")
            break
        
        # Display recommendations
        print(f"\nRecommendations for '{seed_movie_title}':")
        for idx, (movie_title, similarity) in enumerate(recommended_movies, start=1):
            print(f"{idx}. {movie_title} (Similarity: {similarity})")
        
        # Ask for user selection
        selected_index = int(input("\nSelect a movie from the list above by entering its number: ")) - 1
        
        if selected_index < 0 or selected_index >= len(recommended_movies):
            print("Invalid selection. Please enter a valid number.")
            continue
        
        selected_movie_title = recommended_movies[selected_index][0]
        
        # Remove selected movie from S
        selected_movie_id = df[df['title'] == selected_movie_title]['movieID'].values[0]
        S.remove(selected_movie_id)
        
        # Remove all non-selected movies from S
        non_selected_movie_ids = {df[df['title'] == movie[0]]['movieID'].values[0] for idx, movie in enumerate(recommended_movies) if idx != selected_index}
        S.difference_update(non_selected_movie_ids)
        
        # Ask for continuation
        continue_choice = input("\nDo you want to continue? (yes/no): ")
        
        if continue_choice.lower() == 'no':
            print(f"\nYou selected: {selected_movie_title}")
            break
        else:
            # Use the selected movie as the new seed movie
            seed_movie_title = selected_movie_title

# Start the interactive recommendation system
interactive_recommendation_system()


Enter the title of a movie to start with: The Godfather

Recommendations for 'The Godfather':
1. Goodfellas (Similarity: 0.42857142857142855)
2. Carlito's Way (Similarity: 0.25)
3. Scarface (Similarity: 0.25)
4. Casino (Similarity: 0.1111111111111111)
5. A Bronx Tale (Similarity: 0.1111111111111111)

Select a movie from the list above by entering its number: 3

Do you want to continue? (yes/no): yes

Recommendations for 'Scarface':
1. Scarface (Similarity: 1.0)
2. True Romance (Similarity: 0.1111111111111111)
3. Romeo + Juliet (Similarity: 0.1111111111111111)
4. The Freshman (Similarity: 0.1111111111111111)
5. Miller's Crossing (Similarity: 0.1111111111111111)

Select a movie from the list above by entering its number: 3

Do you want to continue? (yes/no): yes

Recommendations for 'Romeo + Juliet':
1. Romeo + Juliet (Similarity: 1.0)
2. A History of Violence (Similarity: 0.42857142857142855)
3. The Juror (Similarity: 0.25)
4. Witness (Similarity: 0.25)
5. The Cider House Rules (Similar

In [42]:
import numpy as np
import pandas as pd

# Assuming df is your DataFrame loaded from "finalDataset.csv"
# df = pd.read_csv("finalDataset.csv")

def jaccard_similarity(keywords1, keywords2):
    set1 = set(keywords1.split(','))
    set2 = set(keywords2.split(','))
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

def recommend_movies_by_title(seed_movie_title, top_n=5, S=set(), weights={}):
    # Find the seed movie's ID
    seed_movie_id = df[df['title'] == seed_movie_title]['movieID'].values[0]
    
    # Find the seed movie's keywords
    seed_movie_keywords = df[df['movieID'] == seed_movie_id]['keywords'].values[0]
    
    # Calculate similarity for each movie in S, excluding the seed movie
    similarities = []
    for movie_id in S:
        if movie_id != seed_movie_id:  # Ensure the seed movie is not considered
            movie_keywords = df[df['movieID'] == movie_id]['keywords'].values[0]
            base_similarity = jaccard_similarity(seed_movie_keywords, movie_keywords)
            # Adjust similarity based on weight
            adjusted_similarity = base_similarity + weights.get(movie_id, 0)  # Default weight is 0
            if base_similarity > 0.03:  # Adjusted to correct threshold
                similarities.append((movie_id, adjusted_similarity))
    
    # Sort by adjusted similarity and get the top N
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Fetch the movie titles for the recommended movie IDs, ensuring uniqueness
    recommended_movies = []
    seen_titles = set()
    for movie_id, similarity in similarities:
        movie_title = df[df['movieID'] == movie_id]['title'].values[0]
        if movie_title not in seen_titles:
            recommended_movies.append((movie_title, similarity))
            seen_titles.add(movie_title)
            if len(recommended_movies) == top_n:
                break
    
    return recommended_movies

def update_weights(selected_movie_id, S, weights):
    selected_keywords = df[df['movieID'] == selected_movie_id]['keywords'].values[0]
    
    # Calculate similarity of the selected movie with all other movies
    similarity_with_selected = {}
    for movie_id in S:
        if movie_id != selected_movie_id:
            movie_keywords = df[df['movieID'] == movie_id]['keywords'].values[0]
            similarity_with_selected[movie_id] = jaccard_similarity(selected_keywords, movie_keywords)
    
    # Calculate mean similarity of selected movie with non-selected movies
    non_selected_similarities = [similarity for movie_id, similarity in similarity_with_selected.items()]
    mean_similarity = np.mean(non_selected_similarities) if non_selected_similarities else 0
    
    # Update weights
    for movie_id, similarity in similarity_with_selected.items():
        delta_wi = similarity - mean_similarity
        weights[movie_id] = weights.get(movie_id, 0) + delta_wi

def interactive_recommendation_system():
    seed_movie_title = input("Enter the title of a movie to start with: ")
    
    # Initialize S with items related to the seed item and weights
    seed_movie_ids = set(df[df['title'] == seed_movie_title]['movieID'].values)
    S = set(df['movieID'].values) - seed_movie_ids  # Exclude all instances of the seed movie
    weights = {movie_id: 0 for movie_id in S}  # Initialize weights with 0
    
    while True:
        recommended_movies = recommend_movies_by_title(seed_movie_title, 5, S, weights)
        
        if not recommended_movies:
            print("No more recommendations available.")
            break
        
        print(f"\nRecommendations for '{seed_movie_title}':")
        for idx, (movie_title, similarity) in enumerate(recommended_movies, start=1):
            print(f"{idx}. {movie_title} (Similarity: {similarity})")
        
        selected_index = int(input("\nSelect a movie from the list above by entering its number: ")) - 1
        if selected_index < 0 or selected_index >= len(recommended_movies):
            print("Invalid selection. Exiting.")
            break
        selected_movie_title = recommended_movies[selected_index][0]
        selected_movie_id = df[df['title'] == selected_movie_title]['movieID'].values[0]
        
        # Update weights based on the selection
        update_weights(selected_movie_id, S, weights)
        
        # Remove only the selected movie from S
        S.remove(selected_movie_id)
        
        continue_choice = input("\nDo you want to continue? (yes/no): ")
        
        if continue_choice.lower() == 'no':
            print(f"\nYou selected: {selected_movie_title}")
            break
        else:
            seed_movie_title = selected_movie_title
            # Ensure the next seed movie is not considered in future recommendations
            seed_movie_ids.add(selected_movie_id)

# Start the interactive recommendation system
interactive_recommendation_system()

Enter the title of a movie to start with: The Godfather

Recommendations for 'The Godfather':
1. Goodfellas (Similarity: 0.42857142857142855)
2. Carlito's Way (Similarity: 0.25)
3. Scarface (Similarity: 0.25)
4. Casino (Similarity: 0.1111111111111111)
5. A Bronx Tale (Similarity: 0.1111111111111111)

Select a movie from the list above by entering its number: 5

Do you want to continue? (yes/no): yes

Recommendations for 'A Bronx Tale':
1. Don't Tell Mom the Babysitter's Dead (Similarity: 0.33090395480225987)
2. Casino (Similarity: 0.21979284369114876)
3. Carlito's Way (Similarity: 0.21979284369114876)
4. In the Name of the Father (Similarity: 0.21979284369114876)
5. The Man Without a Face (Similarity: 0.21979284369114876)

Select a movie from the list above by entering its number: 5

Do you want to continue? (yes/no): yes

Recommendations for 'The Man Without a Face':
1. Don't Tell Mom the Babysitter's Dead (Similarity: 0.49627074580485964)
2. Dirty Dancing (Similarity: 0.3296040791381